In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from Managers.GameDirector import GameDirector
from Bots import RandomBot, AlexPastorBot, EdoBot, AdrianHerasBot
import pandas as pd
from tqdm import tqdm
import random

# bots_configs = [
#     [EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot, RandomBot.RandomBot],
#     [EdoBot.EdoBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot],
#     # [EdoBot.EdoBot, AdrianHerasBot.AdrianHerasBot, AdrianHerasBot.AdrianHerasBot, AdrianHerasBot.AdrianHerasBot],
#     [EdoBot.EdoBot, EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot],
#     [EdoBot.EdoBot, EdoBot.EdoBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot],
#     # [EdoBot.EdoBot, EdoBot.EdoBot, AdrianHerasBot.AdrianHerasBot, AdrianHerasBot.AdrianHerasBot],
#     [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, RandomBot.RandomBot],
#     [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, AlexPastorBot.AlexPastorBot],
#     # [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, AdrianHerasBot.AdrianHerasBot],
#     ]

bots_configs = [
    [EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot, RandomBot.RandomBot],
    # [EdoBot.EdoBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot],
    # [EdoBot.EdoBot, EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot],
    # [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, RandomBot.RandomBot],
    ]

def shuffle_bots(bots):
    names = [f"{bot.__name__}_{j}" for j, bot in enumerate(bots)]
    temp = list(zip(names, bots))
    # random.shuffle(temp)
    res1, res2 = zip(*temp)
    names, bots = list(res1), list(res2)
    return names, bots

def play_single_game(bots):
    names, bots = shuffle_bots(bots)
    game_director = GameDirector(bots = bots)
    game_director.game_start(0)
    last_round = list(game_director.trace_loader.current_trace["game"].values())[-1]
    last_turn = list(last_round.values())[-1]
    points = list(last_turn["end_turn"]["victory_points"].values())
    points = [int(point) for point in points]
    return {name: points[i] for i, name in enumerate(names)}


def play_set_of_games(bots, games_to_play=100):
    return [play_single_game(bots) for i in tqdm(range(games_to_play))]

In [29]:
from joblib import delayed, Parallel

# results_list = Parallel(n_jobs=3)(delayed(play_set_of_games)(bots) for bots in bots_configs)
results_list = [ play_set_of_games(bots) for bots in bots_configs]

for results in results_list:
    df = pd.DataFrame(results)
    max_counts = dict(df.idxmax(axis=1).value_counts())
    bot_wins = {bot: max_counts.get(bot, 0)/len(df.index)*100 for bot in list(df.columns)}

    display(pd.DataFrame(bot_wins, index=["% wins"]).T)

  6%|▌         | 6/100 [02:25<38:05, 24.32s/it]


IndexError: list index out of range

In [21]:
from Bots.helpers import *
from Bots.logger import *
from IPython.display import HTML

clear_log_data()
points = play_single_game([EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot, RandomBot.RandomBot])
winer = max(points, key=points.get)
print(points)
print(winer)

# log_data = get_log_df()
# display_df = log_data[["id","materials", "goal_1", "goal_2", "trade_1", "trade_2", "trade_3", "events"]].head(50)
# display(log_data.goal_1.value_counts())

# display( HTML( display_df.to_html().replace("\\n","<br>") ))

{'AdrianHerasBot_0': 10, 'RandomBot_1': 4, 'RandomBot_2': 2, 'RandomBot_3': 2}
AdrianHerasBot_0


In [22]:
import os
from jsonpath_ng import jsonpath, parse
from jsonpath_ng.ext import parse
import json

folder_path = "traces"
folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
folders.sort()
tmp_path = os.path.join(folder_path, folders[-1],"game_0.json")
jsonpath_expr_mine = parse('game.*.turn_P0.commerce_phase[*].answers[*][*].completed')
jsonpath_expr_total = parse('game.*.*.commerce_phase[*].answers[*][*].completed')

# read file in path and save it in a variable
with open(tmp_path) as f:
    data = json.load(f)
    matches = [match.value for match in jsonpath_expr_mine.find(data)]
    print("mine: ",sum(matches))
    matches = [match.value for match in jsonpath_expr_total.find(data)]
    print("total: ",sum(matches))


mine:  0
total:  0
